In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2

FRAME_W, FRAME_H, FPS = 1280, 720, 30

LOW_YELLOW  = (20, 100, 100)
HIGH_YELLOW = (35, 255, 255)

MIN_AREA_PX   = 1200    
ASPECT_TOL    = 0.35   
SOLIDITY_MIN  = 0.85     
POLY_EPS_FRAC = 0.04     

def mask_yellow(bgr):
   hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
   mask = cv2.inRange(hsv, LOW_YELLOW, HIGH_YELLOW)
   kernel = np.ones((5,5), np.uint8)
   mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
   mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

   return mask

def is_squareish(cnt):
   area = cv2.contourArea(cnt)
   if area < MIN_AREA_PX:
       return False

   x, y, w, h = cv2.boundingRect(cnt)
   ar = w / float(h)
   if not (1 - ASPECT_TOL <= ar <= 1 + ASPECT_TOL):
       return False

   hull = cv2.convexHull(cnt)
   hull_area = cv2.contourArea(hull)
   solidity = (area / hull_area) if hull_area > 0 else 0
   if solidity < SOLIDITY_MIN:
       return False

   peri = cv2.arcLength(cnt, True)
   approx = cv2.approxPolyDP(cnt, POLY_EPS_FRAC * peri, True)
   if len(approx) < 4:
       return False

   return True

def draw_detection(frame, cnt):
   x, y, w, h = cv2.boundingRect(cnt)
   M = cv2.moments(cnt)

   if M["m00"] == 0:
       cx, cy = x + w // 2, y + h // 2
   else:
       cx = int(M["m10"] / M["m00"])
       cy = int(M["m01"] / M["m00"])

   hull = cv2.convexHull(cnt)
   cv2.drawContours(frame, [hull], -1, (255, 255, 0), 2)
   cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
   cv2.circle(frame, (cx,cy), 5, (0,0,255), -1)

   rel_w = w / frame.shape[1]
   rel_h = h / frame.shape[0]
   cv2.putText(frame,
               f"u,v=({cx},{cy})  box~({rel_w:.2f}w,{rel_h:.2f}h)",
               (x, max(0, y-10)),
               cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0,255,0), 2, cv2.LINE_AA)
  
   return hull, (cx, cy)


def main():
   pipeline = rs.pipeline()
   config = rs.config()
  
   config.enable_stream(rs.stream.color, FRAME_W, FRAME_H, rs.format.bgr8, FPS)

   try:
       profile = pipeline.start(config)
       print("RealSense camera started. Press 'q' or ESC to quit.")
   except Exception as e:
       print(f"Failed to start RealSense pipeline: {e}")
       return

   try:
       while True:
           # Get framesi\
           frames = pipeline.wait_for_frames()
           color_frame = frames.get_color_frame()

           if not color_frame:
               continue

           frame = np.asanyarray(color_frame.get_data())
           mask = mask_yellow(frame)
           cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

           out = frame.copy()
           detections = 0
           cube_info = []
          
           for c in cnts:
               if is_squareish(c):
                   hull, centroid = draw_detection(out, c)
                   detections += 1
                  
                   cube_info.append({
                       'hull': hull,
                       'centroid': centroid
                   })
          
           if cube_info:
               print(f"\n=== Frame: {detections} cube(s) detected ===")
               for i, info in enumerate(cube_info):
                   hull_points = info['hull'].reshape(-1, 2)
                   centroid = info['centroid']
                   print(f"Cube {i+1}:")
                   print(f"  Position (centroid): ({centroid[0]}, {centroid[1]})")
                   print(f"  Convex hull points ({len(hull_points)} points):")
                   for j, pt in enumerate(hull_points):
                       print(f"    Point {j+1}: ({pt[0]}, {pt[1]})")

           cv2.putText(out, f"Yellow cubes (2D): {detections}",
                       (12, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2, cv2.LINE_AA)
          
           y_offset = 60
           for i, info in enumerate(cube_info):
               centroid = info['centroid']
               text = f"Cube {i+1}: ({centroid[0]}, {centroid[1]})"
               cv2.putText(out, text, (12, y_offset),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
               y_offset += 30


           cv2.imshow("RealSense (detections)", out)
           cv2.imshow("Yellow mask", mask)


           key = cv2.waitKey(1) & 0xFF
           if key in (27, ord('q')): 
               break


   finally:
       pipeline.stop()
       cv2.destroyAllWindows()


if __name__ == "__main__":
   main()

